# Election Statistics ANALYSIS

## Importing Libraries

In [1]:
from pandas import DataFrame,read_csv,concat,options,set_option,to_datetime,to_numeric
from os import listdir

In [2]:
options.display.float_format = "{:,.2f}".format
set_option('display.max_rows',50)

## National/State Summaries

### Load Data

In [3]:
csv_files:list[str] = listdir('State_Summary_Results')
data:dict[str,DataFrame] = {}
df:DataFrame = DataFrame(['State','KH_Vote_Count', 'KH_Vote_Pct', 'DT_Vote_Count', 'DT_Vote_Pct','Pct_Reported'])

In [4]:
for file in csv_files:
    if('FINAL' in file):
        data[file.replace('_Results.csv','').replace('FINAL_','')] = read_csv(f"State_Summary_Results/{file}")

### Merge state summaries into one giant national table

In [ ]:
for key,item in data.items():
    item.insert(loc=1,column='State',value=key)
    df:DataFrame = concat([df,item],axis=0)

In [ ]:
all_states_summary_df = df.drop([0],axis=1).reset_index(drop=True).iloc[6:]

### Add Nationwide Total Vote Count Columns

In [ ]:
total_votes:DataFrame = all_states_summary_df.groupby('Datetime_Extraction').sum(numeric_only=True)

In [ ]:
total_votes['Nationwide_Total_Votes'] = total_votes['KH_Vote_Count']+total_votes['DT_Vote_Count']
total_votes['Additional_Votes_Since_Last_Update'] = total_votes['Nationwide_Total_Votes'].diff()
#total_votes['Average_Additional_Votes_Per_Minute'] = total_votes['Additional_Votes_Since_Last_Update'].fillna(0).astype(float)/to_datetime(total_votes.index.to_series()).diff()

In [ ]:
vote_updates:DataFrame = total_votes.reset_index(drop=False).groupby('Nationwide_Total_Votes').first().reset_index()[['Datetime_Extraction','Nationwide_Total_Votes','KH_Vote_Count','DT_Vote_Count']]

In [ ]:
vote_updates['Vote_Change'] = vote_updates['Nationwide_Total_Votes'].diff().fillna(0)

In [ ]:
total_votes[total_votes['Additional_Votes_Since_Last_Update']>0][['KH_Vote_Count','DT_Vote_Count','Nationwide_Total_Votes','Additional_Votes_Since_Last_Update']].to_csv('ANALYSIS/Nationwide_Vote_Count_Updates.csv',index=True)

In [ ]:
total_votes[total_votes['Additional_Votes_Since_Last_Update']>0][['KH_Vote_Count','DT_Vote_Count','Nationwide_Total_Votes','Additional_Votes_Since_Last_Update']].info()

### Group State Summaries By Vote Count Changes

In [24]:
for key,item in data.items():
    temp_df:DataFrame = item.copy()
    temp_df.insert(5,'Total_Votes',temp_df['KH_Vote_Count']+temp_df['DT_Vote_Count'])
    temp_df:DataFrame = temp_df.groupby('Total_Votes').first().reset_index(drop=False)
    temp_df['Additional_Votes_Since_Last_Update'] = temp_df['Total_Votes'].diff()
    temp_df['Additional_Trump_Votes_Since_Last_Update'] = temp_df['DT_Vote_Count'].diff()
    temp_df['Additional_Harris_Votes_Since_Last_Update'] = temp_df['KH_Vote_Count'].diff()
    temp_df['Pct_New_Votes_For_Trump'] = (temp_df['Additional_Trump_Votes_Since_Last_Update']/temp_df['Additional_Votes_Since_Last_Update'])*100
    temp_df['Pct_New_Votes_For_Harris'] = (temp_df['Additional_Harris_Votes_Since_Last_Update']/temp_df['Additional_Votes_Since_Last_Update'])*100
    temp_df:DataFrame = temp_df.fillna(0)
    cols = list(temp_df)
    cols.insert(0, cols.pop(cols.index('Datetime_Extraction')))
    temp_df = temp_df.loc[:, cols]
    for col in temp_df.columns:
        try:
            temp_df[col] = temp_df[col].astype(float)
        except:pass
    temp_df.to_csv(f'ANALYSIS/{key}_Vote_Count_Updates.csv',index=True)